In [13]:
import os
from os.path import join
import sys
import json
import panda_utils as util
from PANDA import PANDA_IMAGE
from ImgSplit import ImgSplit
from ResultMerge import DetResMerge
from pycocotools.coco import COCO
from pprint import pprint

In [62]:
def load_json(path):
    with open(path, 'r') as fp:
        jsonfile = json.load(fp)
    return jsonfile

def save_json(path, d):
    with open(path, 'w') as fp:
        json_str = json.dumps(d)
        fp.write(json_str)
    print('Done, save to {}'.format(path))

In [2]:
data_root_dir = 'C:/Users/KV_Zhao/Documents/DATASETS/PANDA/'
image_root = join(data_root_dir, 'image_train')
groundtruth_file_path = join(data_root_dir, 'image_annos/person_bbox_train.json')
output_dir = 'split_single_image'
detection_result_file_path = 'ctdet_800x800_crowdhuman_from_extend_pt_warmup_0404_model_last.json'

In [3]:
output_anno_path = os.path.join(output_dir, 'split.json')
os.makedirs(output_dir, exist_ok=True)
example = PANDA_IMAGE(image_root, groundtruth_file_path, annomode='person')

Loading annotation json file: C:/Users/KV_Zhao/Documents/DATASETS/PANDA/image_annos/person_bbox_train.json


In [4]:
example.showAnns(range=1, shuffle=True)

filename: C:/Users/KV_Zhao/Documents/DATASETS/PANDA/image_train\06_University_Campus/IMG_06_05.jpg


In [5]:
splitter = ImgSplit(image_root, groundtruth_file_path, 'person', output_dir, output_anno_path)

Loading annotation json file: C:/Users/KV_Zhao/Documents/DATASETS/PANDA/image_annos/person_bbox_train.json


In [43]:
sample_image_path = '01_University_Canteen/IMG_01_01.jpg'
patch_annos = splitter.SplitSingle(sample_image_path, 0.5)

filename: C:/Users/KV_Zhao/Documents/DATASETS/PANDA/image_train\01_University_Canteen/IMG_01_01.jpg


In [58]:
def restrain_between_0_1(values_list):
    return_list = []
    for value in values_list:
        if value < 0:
            new_value = 0
        elif value > 1:
            new_value = 1
        else:
            new_value = value
        return_list.append(new_value)
    return return_list

def RectDict2List(rectdict, imgwidth, imgheight, scale, mode='tlbr'):
    x1, y1, x2, y2 = restrain_between_0_1([rectdict['tl']['x'], rectdict['tl']['y'],
                                           rectdict['br']['x'], rectdict['br']['y']])
    xmin = int(x1 * imgwidth * scale)
    ymin = int(y1 * imgheight * scale)
    xmax = int(x2 * imgwidth * scale)
    ymax = int(y2 * imgheight * scale)

    if mode == 'tlbr':
        return xmin, ymin, xmax, ymax
    elif mode == 'tlwh':
        return xmin, ymin, xmax - xmin, ymax - ymin

In [56]:
def anno2coco(annos_dict, anno_types=['fbox']):
    CATEGORY = {
    'visible body': 1,
    'full body': 2,
    'head': 3,
    'vehicle': 4
    }

    CATMAP = {
        'fbox': 'full body',
        'vbox': 'visible body',
        'hbox': 'head',
        'vehicle': 'vehicle'
    }
    json_dict = {
    'images': [],
    'annotations': [],
    'categories': [
        {
        'supercategory': 'person',
        'id': CATEGORY[CATMAP[anno]],
        'name': CATMAP[anno],
        } for anno in anno_types
    ]}

    bbox_id = 1
    ignore = 0 

    for imagename, imagedict in annos_dict.items():
        imgwidth = imagedict['image size']['width']
        imgheight = imagedict['image size']['height']
        image_id = imagedict['image id']
        scale = 1.0
        #scale = showwidth / imgwidth
        image_info = {
            'file_name': imagename,
            'height': imgheight,
            'width': imgwidth,
            'id': image_id,
        }
        json_dict['images'].append(image_info)

        for object_dict in imagedict['objects list']:
            objcate = object_dict['category']
            if objcate == 'person':
                #personpose = object_dict['riding type'] if object_dict['pose'] == 'riding' else object_dict['pose']
                if 'fbox' in anno_types:
                    fullrect = RectDict2List(object_dict['rects']['full body'], imgwidth, imgheight, scale, 'tlwh')
                    annotation = {
                            'area': fullrect[2] * fullrect[3],
                            'iscrowd': ignore,
                            'image_id': image_id,
                            'bbox': fullrect,
                            'category_id': CATEGORY[CATMAP['fbox']],
                            'id': bbox_id,
                            'ignore': ignore,
                            'segmentation': [],
                    }
                    json_dict['annotations'].append(annotation)
                    bbox_id += 1

                if 'vbox' in anno_types:
                    visiblerect = RectDict2List(object_dict['rects']['visible body'], imgwidth, imgheight, scale, 'tlwh')
                    annotation = {
                            'area': visiblerect[2] * visiblerect[3],
                            'iscrowd': ignore,
                            'image_id': image_id,
                            'bbox': visiblerect,
                            'category_id': CATEGORY[CATMAP['vbox']],
                            'id': bbox_id,
                            'ignore': ignore,
                            'segmentation': [],
                    }
                    json_dict['annotations'].append(annotation)
                    bbox_id += 1

                if 'hbox' in anno_types:
                    headrect = RectDict2List(object_dict['rects']['head'], imgwidth, imgheight, scale, 'tlwh')
                    annotation = {
                            'area': headrect[2] * headrect[3],
                            'iscrowd': ignore,
                            'image_id': image_id,
                            'bbox': headrect,
                            'category_id': CATEGORY[CATMAP['hbox']],
                            'id': bbox_id,
                            'ignore': ignore,
                            'segmentation': [],
                    }
                    json_dict['annotations'].append(annotation)
                    bbox_id += 1
    return json_dict

In [45]:
all_image_split_jsonfile_path = join(data_root_dir, 'all_image_split/annoJSONs/split.json')
all_image_split = load_json(all_image_split_jsonfile_path)

In [52]:
# Use split.json image_id as primary key
for image_name, imagedict in all_image_split.items():
    if image_name in patch_annos:
        imgid = imagedict['image id']
        patch_annos[image_name]['image id'] = imgid

In [59]:
patch_coco_annos = anno2coco(patch_annos)

In [60]:
patch_coco_annos

{'images': [{'file_name': '01_University_Canteen_IMG_01_01___0.5__0__0.jpg', 'height': 1025, 'width': 2049, 'id': 1}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__924.jpg', 'height': 1025, 'width': 2049, 'id': 2}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__1848.jpg', 'height': 1025, 'width': 2049, 'id': 3}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__2772.jpg', 'height': 1025, 'width': 2049, 'id': 4}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__3696.jpg', 'height': 1025, 'width': 2049, 'id': 5}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__4620.jpg', 'height': 1025, 'width': 2049, 'id': 6}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__5544.jpg', 'height': 1025, 'width': 2049, 'id': 7}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__6468.jpg', 'height': 1025, 'width': 2049, 'id': 8}, {'file_name': '01_University_Canteen_IMG_01_01___0.5__0__6502.jpg', 'height': 1024, 'width': 2049, 'id': 9}, {'file_name

In [63]:
save_json(join(output_dir, 'patch_coco.json'), patch_coco_annos)

Done, save to split_single_image\patch_coco.json
